In [1]:
ReloadProject('deep_learning')

Path c:\Workspace\git\ added to PYTHON_PATH.
Path c:\Workspace\git\deep_learning added to PYTHON_PATH.


Using TensorFlow backend.


notebook_init.py imported and reloaded
reloaded: a3c_impl
forwarded symbol: a3c_impl
reloaded: base
forwarded symbol: base
reloaded: circular_world_env
forwarded symbol: circular_world_env
reloaded: environment_impl
forwarded symbol: environment_impl
reloaded: guided_environments
forwarded symbol: guided_environments
reloaded: gym
forwarded symbol: gym
reloaded: interval_world_env
forwarded symbol: interval_world_env
reloaded: keras
forwarded symbol: keras
reloaded: layers
forwarded symbol: layers
reloaded: logging
forwarded symbol: logging
reloaded: model_builder
forwarded symbol: model_builder
reloaded: models
forwarded symbol: models
reloaded: numpy
forwarded symbol: numpy
reloaded: numpy_util
forwarded symbol: numpy_util
reloaded: optimizers
forwarded symbol: optimizers
reloaded: other_runners
forwarded symbol: other_runners
reloaded: policy_impl
forwarded symbol: policy_impl
reloaded: qfunc_impl
forwarded symbol: qfunc_impl
reloaded: runner_extension_impl
forwarded symbol: runner_

In [98]:
states

[[array([[ 0.02318315,  0.58242713,  0.08012372, -0.42810967]]),
  array([[ 0.03937273,  0.00450853, -0.04028056, -0.01137044]]),
  array([[-1.22393435, -0.94591479, -0.10243092, -0.30642735]]),
  array([[-1.00146948, -0.89419425, -0.05840249, -0.10293925]]),
  array([[1.26898112, 0.90985765, 0.05516422, 0.0211102 ]]),
  array([[ 0.07734736,  0.38238581,  0.00808368, -0.43614473]]),
  array([[ 0.32898555,  0.54686486,  0.01403588, -0.30861282]]),
  array([[ 0.55113367,  0.40094633,  0.01053951, -0.12118495]]),
  array([[ 0.2304714 ,  0.34016697,  0.01694664, -0.02476815]]),
  array([[ 0.81349645,  1.31489248,  0.14548658, -0.00726391]]),
  array([[-0.50330022,  0.36431455, -0.05797559, -0.07904508]]),
  array([[ 1.29932225,  1.26296664,  0.08618098, -0.25595331]]),
  array([[ 0.07301362,  0.34381018, -0.01331721, -0.5010894 ]])],
 [array([[0.77688442, 0.18271239, 0.01830044, 0.45438987]]),
  array([[ 0.07219074, -0.21979337, -0.0127479 ,  0.3597516 ]]),
  array([[-0.01872194, -0.017809

In [99]:
use_states_1 = states[0][:5]
use_states_2 = states[1][:5]

In [6]:
use_states_1 = [
    np.array([[ 0.02318315,  0.58242713,  0.08012372, -0.42810967]]),
    np.array([[ 0.03937273,  0.00450853, -0.04028056, -0.01137044]]),
    np.array([[-1.22393435, -0.94591479, -0.10243092, -0.30642735]]),
    np.array([[-1.00146948, -0.89419425, -0.05840249, -0.10293925]]),
    np.array([[1.26898112, 0.90985765, 0.05516422, 0.0211102 ]])]

In [5]:
use_states_2 = [
    np.array([[0.77688442, 0.18271239, 0.01830044, 0.45438987]]),
    np.array([[ 0.07219074, -0.21979337, -0.0127479 ,  0.3597516 ]]),
    np.array([[-0.01872194, -0.01780948,  0.03422948,  0.07417699]]),
    np.array([[-0.03013217,  0.19765872,  0.05044142, -0.19798146]]),
    np.array([[ 0.04643257, -0.19702157, -0.01337837,  0.30328672]])]

## A3C

In [7]:
running_environment.ForceCpuForTheRun()

In [10]:
batch_size = 64  # used in qfunc and runner.
envs = [
    environment_impl.GymEnvironment(gym.make('CartPole-v0')),
    environment_impl.GymEnvironment(gym.make('CartPole-v0')),
    environment_impl.GymEnvironment(gym.make('CartPole-v0')),
    environment_impl.GymEnvironment(gym.make('CartPole-v0')),
    environment_impl.GymEnvironment(gym.make('CartPole-v0')),
]
brain = a3c_impl.A3C(
    model=a3c_impl.CreateModel(
        state_shape=env.GetStateShape(),
        action_space_size=env.GetActionSpaceSize(),
        hidden_layer_sizes=(12,),
    )
)

[20:40:24] c:\Workspace\git\deep_learning\engine\a3c_impl.py:46 WARNING: only one A3C instance can be active; the previous instance <deep_learning.engine.a3c_impl.A3C object at 0x00000171DA675F60> is now deactivated.


In [13]:
%%time
logging.ENV.debug_verbosity = 6
runner = other_runners.MultiEnvironmentRunner()
runner.AddCallback(runner_extension_impl.ProgressTracer(report_every_num_of_episodes=20))
runner.AddCallback(runner_extension_impl.ValueTracer(trace_states=use_states_1 + use_states_2, trace_actions=[0, 1]))

Wall time: 0 ns


In [ ]:
%%time
runner.Run(envs=envs, brain=brain, policy=policy_impl.GreedyPolicyWithRandomness(epsilon=0.1), num_of_episodes=20000)

[20:41:11] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 20/20000: avg_reward = 49.45, avg_steps=49.45 (over 20 episodes)
[20:41:12] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 40/20000: avg_reward = 50.55, avg_steps=50.55 (over 20 episodes)
[20:41:13] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 60/20000: avg_reward = 49.60, avg_steps=49.60 (over 20 episodes)
[20:41:14] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 80/20000: avg_reward = 48.60, avg_steps=48.60 (over 20 episodes)
[20:41:15] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 100/20000: avg_reward = 57.85, avg_steps=57.85 (over 20 episodes)
[20:41:16] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 120/20000: avg_reward = 56.40, avg_steps=56.40 (over 20 episodes)
[20:41:17] c:\Workspace\git\deep_learning\engine\runner_extension_impl.py:58 Episode 140/20000: av